Arquivo de auxílio para implementação de códigos responsáveis por auxiliar na documentação do repositório

In [1]:
# Importando bibliotecas
import pandas as pd
import json
import os

In [101]:
# Definindo variáveis
DATA_PATH = '../data'

# Metadados da base activity-data
ACTIVITY_DATA_METADATA = [
    'Tempo de chegada do usuário',
    'Tempo de criação da medição sensorial',
    'Identificação do dispositivo de medição',
    'Índice do dispositivo de medição',
    'Modelo do dispositivo de medição',
    'Identificação do usuário',
    'Categoria',
    'Coordenada x da medição do usuário',
    'Coordenada y da medição do usuário',
    'Coordenada z da medição do usuário'
]

# Metadados da base airbnb
AIRBNB_METADATA = 'A definir'

# Metadados da base bike/station
BIKE_STATION_METADATA = 'A definir'

# Definindo dicionário geral de metadados
METADATA_DICT = {
    'activity-data': ACTIVITY_DATA_METADATA,
    'airbnb': AIRBNB_METADATA,
    'bike-data/station': BIKE_STATION_METADATA
}

In [41]:
# Obtendo tamanho e quantidade de arquivos de um diretório
def get_data_dir_info(data_path=DATA_PATH):
    # Iterando sobre diretórios de dados na pasta raíz
    df_data_info = pd.DataFrame()
    for dir in [d for d in os.listdir(data_path) if '.' not in d]:
        # Extraindo iinformações do diretório
        dir_path = os.path.join(data_path, dir)
        qtd_files = len(os.listdir(dir_path))
        nbytes = sum(d.stat().st_size for d in os.scandir(dir_path) if d.is_file())
        nkilos = round(nbytes / (1024), 2)
        nmegas = round(nbytes / (1024 * 1024), 2)
        ngigas = round(nbytes / (1024 * 1024 * 1024), 2)
        
        # Appendando no DataFrame
        row_info = {
            'dir': dir,
            'qtd_files': qtd_files,
            'nbytes': nbytes,
            'nkilos': nkilos,
            'nmegas': nmegas,
            'ngigas': ngigas
        }
        df_data_info = df_data_info.append(row_info, ignore_index=True)

    return df_data_info

df_data_info = get_data_dir_info()
df_data_info

,dir,qtd_files,nbytes,nkilos,nmegas,ngigas
0,activity-data,80.0,1.192764e+09,1164808.76,1137.51,1.11
1,airbnb,1.0,3.423464e+07,33432.26,32.65,0.03
2,bike-data,2.0,0.000000e+00,0.00,0.00,0.00
3,blogs,1.0,9.210000e+02,0.90,0.00,0.00
4,employee,4.0,0.000000e+00,0.00,0.00,0.00
5,flights-data,3.0,0.000000e+00,0.00,0.00,0.00
6,iot-devices,1.0,6.270722e+07,61237.52,59.80,0.06
7,loans,1.0,1.646310e+05,160.77,0.16,0.00
8,retail-data,1.0,4.558067e+07,44512.37,43.47,0.04
9,sf-fire,2.0,0.000000e+00,0.00,0.00,0.00


___

In [141]:
# Função para ler amostra de dados
def read_sample(file_path, file_ext):

    # Lendo arquivo json
    if file_ext == '.json':
        with open(file_path, 'rb') as f:
            sample = f.readline().decode()
            json_sample = json.loads(sample)

        # Transformando json em DataFrame
        try:
            df_sample = pd.DataFrame(json_sample, index=['value']).T
        except ValueError as ve:
            # Validando presença de arrays e listas no JSON
            for k, v in json_sample.items():
                if type(v) == list:
                    json_sample[k] = '[' + ', '.join(v) + ']'
            # Tentando novamente leitura após transformação
            df_sample = pd.DataFrame(json_sample, index=['value']).T

    # Lendo arquivo csv
    elif file_ext == '.csv':
        df_sample = pd.read_csv(file_path, nrows=1).T
        df_sample.columns = ['value']
        
    # Resetando index e padronizando DataFrame
    df_sample.reset_index(drop=False, inplace=True)
    
    # Enriquecendo DataFrame
    df_sample['dtype'] = [type(row) for row in df_sample['value'].values]
    df_sample.columns = ['Coluna', 'Exemplo', 'Tipo Primitivo']

    return df_sample

# Função para transformar schema de amostra em código markdown de tabela para doc
def transform_schema_to_markdown_table(sample_file_path, metadata):

    # Extraindo tipo primitivo para validação da leitura
    file_ext = os.path.splitext(os.path.basename(sample_file_path))[-1]

    # Obtendo amostra em formato DataFrame
    df_sample = read_sample(file_path=sample_file_path, file_ext=file_ext)

    # Adicionando descrição
    df_sample['Descrição'] = metadata

    # Reordenando colunas
    cols_order = ['Coluna', 'Descrição', 'Tipo Primitivo', 'Exemplo']
    df_sample = df_sample.loc[:, cols_order]

    # Transformando colunas em markdown
    header_mkdown = ''.join([' | ' + c for c in df_sample.columns])[1:] + ' |'
    header_sec_mkdown = ''.join([' | :---:'] * len(df_sample.columns))[1:] + ' |'

    # Transformando registros em markdown
    row_mkdown_str = []
    for idx, row in df_sample.iterrows():
        row_mkdown_str.append(''.join([' | ' + str(v) for v in dict(row).values()])[1:] + ' |')
    row_mkdown = '\n'.join(row_mkdown_str)

    # Unindo headers com registros
    table_mkdown = header_mkdown + '\n' + header_sec_mkdown + '\n' + row_mkdown

    return table_mkdown

# Função para iterar sobre todo o conjunto de dados armazenado
def generate_markdown_metadata(data_path, metadata_dict):
    # Gerando lista de diretórios/bases
    data_folders = [root for root, _, _ in os.walk(data_path)][1:5]

    # Iterando sobre diretórios de arquivos
    mkdown_table_dict = {}
    for dir in data_folders:
        # Extraindo caminho do diretório
        dir_path = os.path.join(data_path, dir)
        sample = os.path.join(dir_path, os.listdir(dir_path)[0]) # Validar se este elemento é um diretório. Se sim, iterar nele e pegar amostras

        # Verificando se o diretório contém subdiretórios
        if os.path.isdir(sample):
            pass
        else:
            # Preparando chave de identificação do diretório
            dict_key = dir_path.replace(data_path, '').replace('\\', '/')[2:]

            # Extarindo metadados
            try:
                metadata = metadata_dict[dict_key]
            except KeyError as ke:
                metadata = 'Informação não encontrada'

            # Enriquecendo direcionário geral de metadados em markdown
            mkdown_table_dict[dict_key] = transform_schema_to_markdown_table(sample_file_path=sample, metadata=metadata)

    return mkdown_table_dict
    

___

In [142]:
# Testando execução em loop
DATA_FOLDERS = [root for root, _, _ in os.walk(DATA_PATH)][1:7]

# Iterando sobre diretórios de arquivos
mkdown_table_dict = {}
for dir in DATA_FOLDERS:
    # Extraindo caminho do diretório
    dir_path = os.path.join(DATA_PATH, dir)
    sample = os.path.join(dir_path, os.listdir(dir_path)[0]) # Validar se este elemento é um diretório. Se sim, iterar nele e pegar amostras
    
    # Verificando se o diretório contém subdiretórios
    if os.path.isdir(sample):
        pass
    else:
        # Preparando chave de identificação do diretório
        dict_key = dir_path.replace(DATA_PATH, '').replace('\\', '/')[2:]

        # Extarindo metadados
        try:
            metadata = METADATA_DICT[dict_key]
        except KeyError as ke:
            metadata = 'Informação não encontrada'

        # Enriquecendo direcionário geral de metadados em markdown
        mkdown_table_dict[dict_key] = transform_schema_to_markdown_table(sample_file_path=sample, metadata=metadata)

In [143]:
mkdown_table_dict.keys()

dict_keys(['activity-data', 'airbnb', 'bike-data/station', 'bike-data/trip', 'blogs'])

In [144]:
print(mkdown_table_dict['blogs'])

| Coluna | Descrição | Tipo Primitivo | Exemplo |
| :---: | :---: | :---: | :---: |
| Id | Informação não encontrada | <class 'int'> | 1 |
| First | Informação não encontrada | <class 'str'> | Jules |
| Last | Informação não encontrada | <class 'str'> | Damji |
| Url | Informação não encontrada | <class 'str'> | https://tinyurl.1 |
| Published | Informação não encontrada | <class 'str'> | 1/4/2016 |
| Hits | Informação não encontrada | <class 'int'> | 4535 |
| Campaigns | Informação não encontrada | <class 'str'> | [twitter, LinkedIn] |
